In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
config = {
    'domain': 'cdr',
    'disc_model_class': 'lstm',
    'supervision': 'traditional',
    'max_train': 500,
    'disc_params_default': {
        'n_epochs': 2,
    },
#     'learn_deps': False,
#     'gen_init_params': {
#         'lf_propensity'         : True,
#         'lf_prior'              : False, # Always False 
#         'class_prior'           : True,
#         'lf_class_propensity'   : True,
#         'seed'                  : 646445,
#     },
#     'max_explanations': 10,
#     'gen_model_search_space': 10,
#     'apply_filters': True,
#     'gen_params_default': {
#         'step_size'     : 0.0001,
#         'decay'         : 0.90,
#         'reg_param'     : 0.50,
#         'epochs'        : 25,
#     },
    'gen_model_search_space': 20,
    'disc_model_search_space': 1,
}

In [ ]:
# Get DB connection string and add to globals
# NOTE: $SNORKELDB must be set before any snorkel imports
import os

default_db_name = 'babble_' + config['domain'] + ('_debug' if config.get('debug', False) else '')
DB_NAME = config.get('db_name', default_db_name)
if 'postgres' in config and config['postgres']:
    DB_TYPE = 'postgres'
else:
    DB_TYPE = 'sqlite'
    DB_NAME += '.db'
DB_ADDR = "localhost:{0}".format(config['db_port']) if 'db_port' in config else ""
os.environ['SNORKELDB'] = '{0}://{1}/{2}'.format(DB_TYPE, DB_ADDR, DB_NAME)
print("$SNORKELDB = {0}".format(os.environ['SNORKELDB']))

In [ ]:
from snorkel import SnorkelSession
session = SnorkelSession()

# Resolve config conflicts (nb_config > local_config > global_config)
from snorkel.contrib.babble.pipelines import merge_configs, get_local_pipeline
config = merge_configs(config)

if config['debug']:
    print("NOTE: --debug=True: modifying parameters...")
    config['max_docs'] = 100
    config['gen_model_search_space'] = 1
    config['disc_model_search_space'] = 1
    config['gen_params_default']['epochs'] = 25
    config['disc_params_default']['n_epochs'] = 5

from snorkel.models import candidate_subclass
candidate_class = candidate_subclass(config['candidate_name'], config['candidate_entities'])

pipeline = get_local_pipeline(config['domain'])
pipe = pipeline(session, candidate_class, config)

In [ ]:
# %time pipe.parse()

In [ ]:
# %time pipe.extract()

In [ ]:
# %time pipe.load_gold()

In [ ]:
# %time pipe.featurize()

In [ ]:
# %time pipe.collect()

In [ ]:
# %time pipe.label()

In [ ]:
# %time pipe.supervise()

In [ ]:
%time pipe.classify()